In [1]:
import mysql.connector as sql
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [2]:
db_connection = sql.connect(host='localhost', database='benedict', user='root', password='artema2')

In [3]:
df = pd.read_sql('SELECT * FROM sepsis_case_events;', con=db_connection)

In [4]:
df.head()

,fin,event_date,event_name,result_val,result_unit,event_category,route
0,ZcCVvgGl2L0T,05/14/2014 05:56,Diastolic BP,87.0,mmHg,vitals,
1,ZcCVvgGl2L0T,05/14/2014 07:08,Diastolic BP,67.0,mmHg,vitals,
2,ZcCVvgGl2L0T,05/14/2014 09:11,Diastolic BP,64.0,mmHg,vitals,
3,ZcCVvgGl2L0T,05/14/2014 09:44,Diastolic BP,78.0,mmHg,vitals,
4,ZcCVvgGl2L0T,05/14/2014 10:25,Diastolic BP,62.0,mmHg,vitals,


In [5]:
df['event_name'].value_counts()

O2 Saturation                933498
Pulse                        908968
Respiratory Rate             787765
Systolic BP                  665662
Diastolic BP                 664535
Temperature Conversion       227040
Epinephrine                  213577
Temperature Metric           212008
Norepinephrine               194485
Normal Saline                 92634
Sodium                        74657
Anion Gap                     73892
Potassium                     72892
CO2                           72681
Chloride                      72622
Creatinine                    71344
BUN                           71336
Glucose                       66896
Vasopressin                   55311
D5W                           53321
Magnesium                     52351
Phosphate                     46905
Phenylephrine                 38852
Piperacillin-Tazobactam       38389
Vancomycin                    35335
D5 Half Saline                33587
Lactate                       25617
Bilirubin                   

In [6]:
# Drop 'Temperature Metric'

df = df.query("event_name != 'Temperature Metric'")

In [7]:
# Normal value ranges
df['result_val_category'] = ''
# O2 Saturation 
df.loc[(df['event_name'] == 'O2 Saturation') & (df['result_val'] >= 96), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'O2 Saturation') & (df['result_val'] < 96), 'result_val_category'] = 'low'
# Pulse (60 to 100 bpm)
df.loc[(df['event_name'] == 'Pulse') & (df['result_val'] < 60), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Pulse') & (df['result_val'] >= 60) & (df['result_val'] <=100), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Pulse') & (df['result_val'] > 100), 'result_val_category'] = 'high'


# Respiratory Rate (12 to 25)
df.loc[(df['event_name'] == 'Respiratory Rate') & (df['result_val'] < 12), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Respiratory Rate') & (df['result_val'] >= 12) & (df['result_val'] <=25), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Respiratory Rate') & (df['result_val'] > 25), 'result_val_category'] = 'high'


# Systolic BP 
df.loc[(df['event_name'] == 'Systolic BP') & (df['result_val'] < 90), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Systolic BP') & (df['result_val'] >= 90) & (df['result_val'] <=120), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Systolic BP') & (df['result_val'] >= 121) & (df['result_val'] <=140), 'result_val_category'] = 'prehypertension'
df.loc[(df['event_name'] == 'Systolic BP') & (df['result_val'] > 140), 'result_val_category'] = 'hypertension'


# Diastolic BP                 
df.loc[(df['event_name'] == 'Diastolic BP') & (df['result_val'] < 60), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Diastolic BP') & (df['result_val'] >= 60) & (df['result_val'] <=80), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Diastolic BP') & (df['result_val'] >= 81) & (df['result_val'] <=90), 'result_val_category'] = 'prehypertension'
df.loc[(df['event_name'] == 'Diastolic BP') & (df['result_val'] > 90), 'result_val_category'] = 'hypertension'


# Temperature Conversion    
df.loc[(df['event_name'] == 'Temperature Conversion') & (df['result_val'] <= 37), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Temperature Conversion') & (df['result_val'] > 37) & (df['result_val'] <=38), 'result_val_category'] = 'elevated'
df.loc[(df['event_name'] == 'Temperature Conversion') & (df['result_val'] > 38), 'result_val_category'] = 'high'

# Sodium (135 to 145)
df.loc[(df['event_name'] == 'Sodium') & (df['result_val'] < 135), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Sodium') & (df['result_val'] >= 135) & (df['result_val'] <=145), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Sodium') & (df['result_val'] > 145), 'result_val_category'] = 'high'

# Anion Gap (8-16)
df.loc[(df['event_name'] == 'Anion Gap') & (df['result_val'] < 8), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Anion Gap') & (df['result_val'] >= 8) & (df['result_val'] <=16), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Anion Gap') & (df['result_val'] > 16), 'result_val_category'] = 'high'

# Potassium (3.5-5.0)
df.loc[(df['event_name'] == 'Potassium') & (df['result_val'] < 3.5), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Potassium') & (df['result_val'] >= 3.5) & (df['result_val'] <=5.0), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Potassium') & (df['result_val'] >= 5.1) & (df['result_val'] <=6.0), 'result_val_category'] = 'mild hyperkalemia'
df.loc[(df['event_name'] == 'Potassium') & (df['result_val'] >= 6.1) & (df['result_val'] <=7.0), 'result_val_category'] = 'moderate hyperkalemia'
df.loc[(df['event_name'] == 'Potassium') & (df['result_val'] > 7), 'result_val_category'] = 'severe hyperkalemia'

# CO2 (23 to 29)
df.loc[(df['event_name'] == 'CO2') & (df['result_val'] < 23), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'CO2') & (df['result_val'] >= 23) & (df['result_val'] <=29), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'CO2') & (df['result_val'] > 29), 'result_val_category'] = 'high'


# Chloride (96 and 106)
df.loc[(df['event_name'] == 'Chloride') & (df['result_val'] < 96), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Chloride') & (df['result_val'] >= 96) & (df['result_val'] <=106), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Chloride') & (df['result_val'] > 106), 'result_val_category'] = 'high'

# Creatinine (0.6 to 1.2)
df.loc[(df['event_name'] == 'Creatinine') & (df['result_val'] < 0.6), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'Creatinine') & (df['result_val'] >= 0.6) & (df['result_val'] <=1.2), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Creatinine') & (df['result_val'] > 1.2), 'result_val_category'] = 'high'

# BUN (7 to 20)
df.loc[(df['event_name'] == 'BUN') & (df['result_val'] < 7), 'result_val_category'] = 'low'
df.loc[(df['event_name'] == 'BUN') & (df['result_val'] >= 7) & (df['result_val'] <=20), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'BUN') & (df['result_val'] > 20), 'result_val_category'] = 'high'

# Glucose ( < 100 mg/dL = normal. 100 to 125 = pre-diabetes)
df.loc[(df['event_name'] == 'Glucose') & (df['result_val'] < 100), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Glucose') & (df['result_val'] >= 100) & (df['result_val'] <=125), 'result_val_category'] = 'prediabetes'
df.loc[(df['event_name'] == 'Glucose') & (df['result_val'] > 125), 'result_val_category'] = 'diabetes'

In [8]:
# Medications

# Epinephrine                  
df.loc[(df['event_name'] == 'Epinephrine'), 'result_val_category'] = 'Y'
# Norepinephrine
df.loc[(df['event_name'] == 'Norepinephrine'), 'result_val_category'] = 'Y'
# Normal Saline                 
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Vasopressin                   
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# D5W                           
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Phenylephrine                 
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Piperacillin-Tazobactam       
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Vancomycin                    
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# D5 Half Saline                
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Lactate                       
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Lactated Ringers              
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Metronidazole                 
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# D10                           
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Meropenem                     
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Normal saline (for bolus)      
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Half Saline                    
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Levofloxacin                    
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'
# Plasmalyte A (for bolus)        
df.loc[(df['event_name'] == 'Normal Saline'), 'result_val_category'] = 'Y'


In [9]:


# Glucose ( < 100 mg/dL = normal. 100 to 125 = pre-diabetes)
df.loc[(df['event_name'] == 'Glucose') & (df['result_val'] < 100), 'result_val_category'] = 'normal'
df.loc[(df['event_name'] == 'Glucose') & (df['result_val'] >= 100) & (df['result_val'] <=125), 'result_val_category'] = 'prediabetes'
df.loc[(df['event_name'] == 'Glucose') & (df['result_val'] > 125), 'result_val_category'] = 'diabetes'

df.query("event_name == 'Glucose'")



#                        66896
# Magnesium                     52351
# Phosphate                     46905
# Bilirubin                     25599

# Albumin                       24740
# PaCO2                         24007
# Blood Gas pHa                 22611

# PO2                            4125
# PCO2 Arterial                  3924
# PCO2 Venous                     305

,fin,event_date,event_name,result_val,result_unit,event_category,route,result_val_category
4327172,ZcCVvgGl2L0T,05/14/2014 06:25,Glucose,199.0,mg/dL,labs,,diabetes
4327173,ZcCVvgGl2L0T,05/15/2014 04:41,Glucose,81.0,mg/dL,labs,,normal
4327174,ZcCVvgGl2L0T,05/16/2014 17:28,Glucose,84.0,mg/dL,labs,,normal
4327175,ZcCVvgGl2L0T,05/17/2014 04:28,Glucose,115.0,mg/dL,labs,,prediabetes
4327176,ZcCVvgGl2L0T,05/18/2014 04:13,Glucose,146.0,mg/dL,labs,,diabetes
4327177,ZcCVvgGl2L0T,05/19/2014 04:46,Glucose,108.0,mg/dL,labs,,prediabetes
4327178,TXVDMboxsAIp,06/18/2014 15:18,Glucose,113.0,mg/dL,labs,,prediabetes
4327179,TXVDMboxsAIp,06/21/2014 03:42,Glucose,49.0,mg/dL,labs,,normal
4327180,TXVDMboxsAIp,06/21/2014 05:50,Glucose,75.0,mg/dL,labs,,normal
4327181,TXVDMboxsAIp,06/24/2014 04:09,Glucose,133.0,mg/dL,labs,,diabetes


In [10]:
# Save file with discretized event values
df.to_csv('sepsis_events_categories.csv')

In [14]:
df['event_name'].value_counts()

O2 Saturation                933498
Pulse                        908968
Respiratory Rate             787765
Systolic BP                  665662
Diastolic BP                 664535
Temperature Conversion       227040
Epinephrine                  213577
Norepinephrine               194485
Normal Saline                 92634
Sodium                        74657
Anion Gap                     73892
Potassium                     72892
CO2                           72681
Chloride                      72622
Creatinine                    71344
BUN                           71336
Glucose                       66896
Vasopressin                   55311
D5W                           53321
Magnesium                     52351
Phosphate                     46905
Phenylephrine                 38852
Piperacillin-Tazobactam       38389
Vancomycin                    35335
D5 Half Saline                33587
Lactate                       25617
Bilirubin                     25599
Lactated Ringers            

In [34]:
q = "event_name in ('O2 Saturation','Pulse', 'Respiratory Rate', 'Systolic BP','Diastolic BP','Temperature Conversion','Vasopressin','Epinephrine', 'Norepinephrine')"
df_sub = df.query(q)
df_sub['event_name'].value_counts()

O2 Saturation             933498
Pulse                     908968
Respiratory Rate          787765
Systolic BP               665662
Diastolic BP              664535
Temperature Conversion    227040
Epinephrine               213577
Norepinephrine            194485
Vasopressin                55311
Name: event_name, dtype: int64

In [35]:
# Result values need to be discretized here
# df.groupby(['event_name','result_val'])['result_val'].count()

# df.groupby(['event_name'])['event_name'].count()
#basket = df.groupby(['fin','event_name'])['event_name'].count().unstack().reset_index().fillna(0).set_index('fin')
basket = df_sub.groupby(['fin','event_name'])['event_name'].count().unstack().reset_index().fillna(0).set_index('fin')
basket.head()


event_name,Diastolic BP,Epinephrine,Norepinephrine,O2 Saturation,Pulse,Respiratory Rate,Systolic BP,Temperature Conversion,Vasopressin
fin,,,,,,,,,
+/wwoxFTiWTn,14.0,0.0,0.0,14.0,16.0,13.0,14.0,10.0,0.0
+0sUlZTDUJij,87.0,0.0,0.0,91.0,99.0,87.0,87.0,61.0,0.0
+16f+oeBrbDV,567.0,0.0,0.0,471.0,456.0,321.0,567.0,0.0,0.0
+1HeBTXSNHLh,817.0,212.0,212.0,827.0,832.0,497.0,817.0,75.0,177.0
+2hf8/i2b1dN,231.0,63.0,63.0,359.0,360.0,353.0,231.0,115.0,0.0


In [36]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head()

event_name,Diastolic BP,Epinephrine,Norepinephrine,O2 Saturation,Pulse,Respiratory Rate,Systolic BP,Temperature Conversion,Vasopressin
fin,,,,,,,,,
+/wwoxFTiWTn,1,0,0,1,1,1,1,1,0
+0sUlZTDUJij,1,0,0,1,1,1,1,1,0
+16f+oeBrbDV,1,0,0,1,1,1,1,0,0
+1HeBTXSNHLh,1,1,1,1,1,1,1,1,1
+2hf8/i2b1dN,1,1,1,1,1,1,1,1,0


In [37]:
#basket_random = basket_sets.sample(1000)

frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.998496,(Diastolic BP)
1,0.445113,(Epinephrine)
2,0.431830,(Norepinephrine)
3,1.000000,(O2 Saturation)
4,1.000000,(Pulse)


In [39]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
#rules.head()
rules.shape



(14934, 9)

In [40]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Diastolic BP),(O2 Saturation),0.998496,1.000000,0.998496,1.000000,1.000000,0.000000,inf
1,(O2 Saturation),(Diastolic BP),1.000000,0.998496,0.998496,0.998496,1.000000,0.000000,1.000000
2,(Pulse),(Diastolic BP),1.000000,0.998496,0.998496,0.998496,1.000000,0.000000,1.000000
3,(Diastolic BP),(Pulse),0.998496,1.000000,0.998496,1.000000,1.000000,0.000000,inf
4,(Diastolic BP),(Respiratory Rate),0.998496,1.000000,0.998496,1.000000,1.000000,0.000000,inf
5,(Respiratory Rate),(Diastolic BP),1.000000,0.998496,0.998496,0.998496,1.000000,0.000000,1.000000
6,(Systolic BP),(Diastolic BP),0.998747,0.998496,0.998496,0.999749,1.001255,0.001251,5.992481
7,(Diastolic BP),(Systolic BP),0.998496,0.998747,0.998496,1.000000,1.001255,0.001251,inf
8,(Epinephrine),(Norepinephrine),0.445113,0.431830,0.431830,0.970158,2.246622,0.239617,19.039069
9,(Norepinephrine),(Epinephrine),0.431830,0.445113,0.431830,1.000000,2.246622,0.239617,inf
